In [8]:
import pandas as pd
import bamboolib

from bunkatech.nested_topic_modeling import nested_topic_modeling

path = '/Volumes/OutFriend/citypop'
comments = pd.read_csv(path + '/reddit/comments.csv', sep = '\t')

In [85]:
bunka = nested_topic_modeling().fit(
    comments,
    text_var="body",
    index_var="id",
    sample_size=5000,
    sample_terms=5000,
    embeddings_model="sbert",
    ngrams=(2, 2),
    ents=False,
    language="en",
    db_path=".",
)

Embeddings..
Territory embedding..


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Reducing the vectors..
UMAP(angular_rp_forest=True, metric='cosine', n_components=5, n_neighbors=10, verbose=True)
Sat Feb 19 22:57:13 2022 Construct fuzzy simplicial set
Sat Feb 19 22:57:19 2022 Finding Nearest Neighbors
Sat Feb 19 22:57:20 2022 Finished Nearest Neighbor Search
Sat Feb 19 22:57:20 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Sat Feb 19 22:57:24 2022 Finished embedding
Extract Terms...


100%|██████████████████████████████████████████████████████| 2495/2495 [00:20<00:00, 121.45it/s]


#### Find centroids

In [87]:
df_clusters = bunka.h_clusters_names
emb = bunka.embeddings

In [88]:
df_emb = pd.merge(df_clusters, emb.drop('level_0', axis=1), on = 'id')
new_emb = df_emb.drop(['lemma_0', 'lemma_1', 'lemma_2'], axis=1)

In [89]:
from sklearn.cluster import KMeans
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


def closest_point(centroid, matrix, n=2):
    # Get the n closest points to the centroids
    matrix = np.asarray(matrix)
    dist_2 = np.sum((matrix - centroid)**2, axis=1)
    point = np.argsort(dist_2)[:n]
    return point

def farest_point(centroid, matrix):
    matrix = np.asarray(matrix)
    dist_2 = np.sum((matrix - centroid)**2, axis=1)
    point = np.argsort(dist_2)[-1]
    #point = np.argmax(dist_2)
    return point


def find_centroids(df_fin, top_elements = 4, cluster_var = 'level_0', index_var = 'id'):


    clusters = []
    centroid_elements = []
    far_elements = []
    distances = []

    for cluster in set(df_fin[cluster_var]):
        clusters.append(cluster)

        df_filter = df_fin[df_fin[cluster_var] == cluster]

        # the matrix is the embedding (it can be any embedding)
        matrix = df_filter[[0, 1, 2, 3, 4]].values

        # Compute the centroid
        centroid = np.average(matrix, axis=0)

        # Find the index in the matrix of the clostest point
        closest_index = closest_point(centroid, matrix, n=top_elements)

         # Find the index in the matrix of the farest point
        far_index = farest_point(centroid, matrix)

        # find the distance bewteen the centroid and the farest point
        distance =  np.linalg.norm(matrix[far_index] - matrix[closest_index])
        distances.append(distance)

        # find the corresponding element to the centroid in the metadata    
        centroid_element = df_filter.iloc[closest_index][index_var].tolist()
        if len(centroid_element) >= 1:
            centroid_element = ' || '.join(centroid_element)
        else:
            pass

        centroid_elements.append(centroid_element)


        # find the corresponding element to the farest point in the metadata
        far_element = df_filter.iloc[far_index][index_var]
        far_elements.append(far_element)

    # Create a DataFrame with the elements
    df_centroids = pd.DataFrame({cluster_var:clusters, 
                                 'centroid_element':centroid_elements, 
                                 'far_elements':far_elements, 
                                 'radius':distances})
    return df_centroids

In [90]:
df_centroids = find_centroids(new_emb, 
                              top_elements = 4, 
                              cluster_var = 'level_2', 
                              index_var = 'id')



In [91]:
df_centroids['centroid_element'] = df_centroids['centroid_element'].apply(lambda x : x.split(' || '))
df_centroids = df_centroids.explode('centroid_element')


test = pd.merge(df_centroids, df_clusters, left_on = 'centroid_element', right_on = 'id')
test = pd.merge(test, comments, on = 'id')
test = test[['lemma_2', 'body']]
test

,lemma_2,body
0,Big Wave | Pocket Music | Summer Days,Listen to the album of this. It is also good. ...
1,Big Wave | Pocket Music | Summer Days,It's my second favorite of his after Ride On T...
2,Big Wave | Pocket Music | Summer Days,I like *Weekend Fly to the Sun* because every ...
3,Big Wave | Pocket Music | Summer Days,Enjoy! My favorite track is “Street Dancer”
4,Great Album | album ️ | nice album,So lucky I love love love this album . what a ...
...,...,...
182,Tatsuro Yamashita | OMEGA TRIBE | music ⭐,[tatsuro yamashita live at festival hall](http...
183,watch?v= | Akina Nakamori | MEIKO NAKAHARA,Koi No Projection\tis one of my favs. [here it...
184,watch?v= | Akina Nakamori | MEIKO NAKAHARA,Compare to [Masayoshi Takanaka – Star Wars Sa...
185,watch?v= | Akina Nakamori | MEIKO NAKAHARA,[Lonely Sunset](https://www.youtube.com/watch?...
